In [1]:
# import packages
import os
from PyPDF2 import PdfFileReader
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
# import packages
import os
from PyPDF2 import PdfFileReader
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Define a function to extract text from PDF file
def extract_text_from_pdf(file_path):
    text = textract.process(file_path, method='pdfminer')
    return text.decode('utf-8')

# Define a function to process the text: tokenization, stopword removal
def process_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

folder_path = "../NLP/Articles/"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(folder_path, file_name)
        text = extract_text_from_pdf(file_path)
        processed_text = process_text(text)

TypeError: expected str, bytes or os.PathLike object, not NoneType